<a href="https://colab.research.google.com/github/saiteja-namani-11/chatbot/blob/main/ChatBot_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
#1 Importing the libraries
import numpy as np
import tensorflow as tf
import re
import time
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np
import tensorflow as tf
import re
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import random
import gc
from tensorflow.keras.optimizers import Adam

In [44]:
#2 mounting the drive to the colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
#3 Importing the dataset
lines = open('/content/drive/MyDrive/NLP LAB WORKS/project/Chatbot-Seq2Seq-master/Data/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('/content/drive/MyDrive/NLP LAB WORKS/project/Chatbot-Seq2Seq-master/Data/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
# reads the files including the encoding of the file.
# spliting is performed as its a big text file which returns the list of strings

In [46]:
#4 Creating a dictionary that maps each line and its id
id2line = {}# creatiing an empty dictionary
for line in lines: # a loop that iterates over each line in lines 
    _line = line.split(' +++$+++ ') # splits the line and stored in _line
    if len(_line) == 5: # checks for the length of the list is equal to 5
        id2line[_line[0]] = _line[4] # maps each line id to its corresponding line of dialog in the id2line dictionry

In [47]:
#5 Creating a list of all of the conversations
conversations_ids = [] # empty list which stores the conversation ids parsed from the data.
for conversation in conversations[:-1]: # looping the conversations from the data 
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "") # removing all the quotes spaces, squared brackets from the conversation
    conversations_ids.append(_conversation.split(',')) # spliting the string by commas and appending it into the conversation ids.

In [48]:
#6 Getting separately the questions and the answers
questions = []
answers = []# creating 2 empty lists
for conversation in conversations_ids: # looping on the conversation ids.
    for i in range(len(conversation) - 1):# another loop over the ids from start to last but one id
        questions.append(id2line[conversation[i]]) 
        answers.append(id2line[conversation[i+1]])
        # for every itteration the ith element is considered as question and i+1 as the answer.

In [49]:
#7 Doing a first cleaning of the texts
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;$:<>{}`+=~|.!?,]", "", text)
    return text
    #removing all the punctuations and special characters and converts the text to lower case.

In [50]:
#8 Cleaning the questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    # storing the clean text into a new list

In [51]:
#9 Cleaning the answers
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))
    # storing the clean text into a new list

In [52]:
#10 Filtering out the questions and answers that are too short or too long
short_questions = []
short_answers = []
i = 0
for question in clean_questions:
    if 2 <= len(question.split()) <= 25:# checing for the length greater than or equal to 2 and less than or equal to25
        short_questions.append(question)
        short_answers.append(clean_answers[i])
    i += 1
    # if the condition is passed the corresponding questions and answers are added to the empty list.
clean_questions = []
clean_answers = []
i = 0
for answer in short_answers:
    if 2 <= len(answer.split()) <= 25:
        clean_answers.append(answer)
        clean_questions.append(short_questions[i])
    i += 1
    # the same condition the code is added to the clean questions and clean answers.

In [53]:
#11 Creating a dictionary that maps each word to its number of occurrences
word2count = {} # creating an empty dictionary
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
            # maps each word in the cleaned question to its number of occurance
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
            # maps each word in the cleaned answers to its number of occurance

In [54]:
#12 Creating two dictionaries that map the questions words and the answers words to a unique integer
threshold_questions = 15 # 15 minimum number of occurances a word should have to be included in the dictionary of questions and answers.
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number
        word_number += 1
        # the mapping happens for each question and the total nnumber of words that meet the threshold are collected.
threshold_answers = 15
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number
        word_number += 1
        # the mapping happens for each answer and the total nnumber of words that meet the threshold are collected.

In [55]:
#13 Adding the last tokens to these two dictionaries
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']# new tokents are stored in the tokens variable.
# which include padding of hte sequence which is making sure of the length,end of sequence marker.
#<out> is used for rare words that are not included int he dictionary
#and the start of the sequence marker.
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1
# all of these markers are added to both the question and answer words.

In [56]:
#14 Creating the inverse dictionary of the answerswords2int dictionary
answersints2word = {w_i: w for w, w_i in answerswords2int.items()}
# the new dictionary has the inverse of what answerswords2int had.

In [57]:
#15 Adding the End Of String token to the end of every answer
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>'
    # adding an end of sequence after every answer in clean answers.

In [58]:
#16 Translating all the questions and the answers into integers
# and Replacing all the words that were filtered out by <OUT> 
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
    # the inteeger representation of the questions are stored in the questions_into_int
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)
    # the inteeger representation of the answers are stored in the answers_into_int

In [59]:
#17 Sorting questions and answers by the length of questions
sorted_clean_questions = []
sorted_clean_answers = [] # created empty lists.
for length in range(1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])
# this block of code actually sorts the questions based on their length of the question.

In [60]:
#18 Reduce the threshold for word occurrence
threshold_questions = 10
threshold_answers = 10
# the minimum threshold is dropped from 15 to 10

In [61]:
#19
#Set the parameters for the model
input_vocab_size = len(questionswords2int) + 1 
# the input vocab size is set to the question size +1
output_vocab_size = len(answerswords2int) +1
#the output vocab size is set to the answer size +1
max_length = 25


In [62]:
#20 Define the encoder input
encoder_inputs = Input(shape=(None,))# indicates the model can handle variable length sequence.
encoder_embedding = Embedding(input_vocab_size, 128)(encoder_inputs)
# creating an embedded layer to map the input sequence to a denser vector space with 128
encoder_outputs, state_h, state_c = LSTM(128, return_state=True)(encoder_embedding)
# creating an LSTM layer with 128 units that reeturns the hidden state and the cell state.
encoder_states = [state_h, state_c]

In [63]:
#21 Define the decoder input
decoder_inputs = Input(shape=(None,)) #indicates the model can handle variable length sequence.
decoder_embedding = Embedding(output_vocab_size, 128)(decoder_inputs) 
#Creates an embedding layer for the decoder input.
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
# defines an LSTM layer for the decoder with 128 units 
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

In [64]:
#22 Define the output layerof the decoder which uses dense layers with softmax activation function.
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = Dense(output_vocab_size, activation='softmax')(decoder_outputs)

In [65]:
#23 Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model using the adams optimizer  and a categorical cross entropy loss.
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [66]:
#24 Split the dataset into training and testing sets
import random

sample_size = 10000  # Adjust this value according to your needs
sample_indices = random.sample(range(len(sorted_clean_questions)), sample_size)
# sampling 10000 off the sorted clean questions.

sample_questions = [sorted_clean_questions[i] for i in sample_indices]
sample_answers = [sorted_clean_answers[i] for i in sample_indices]

X_train, X_test, y_train, y_test = train_test_split(sample_questions, sample_answers, test_size=0.1, random_state=42)

In [67]:
#25 Prepare the data for training
from tensorflow.keras.preprocessing.sequence import pad_sequences #using to pad sequences to the same length.
from tensorflow.keras.utils import to_categorical # using to get the onehot encoded
#  Free up memory
del lines, conversations, questions, answers, word2count
# deleting variable line to free up memory
gc.collect()# forces the garbage collector to free up memory

6369

In [68]:
#26 
def preprocess_data(questions, answers, max_length):
    encoder_input_data = pad_sequences(questions, maxlen=max_length, padding='post')#pading zeros at the end of all the sequences have same length
    decoder_input_data = pad_sequences(answers, maxlen=max_length, padding='post')# this is obtained by padding the answers seq with zero
    decoder_target_data = np.zeros((len(answers), max_length, output_vocab_size), dtype='float32') # creating ana array of zeros
    
    for i, answer in enumerate(answers):
        for t, word_int in enumerate(answer):
            if t > 0:
                decoder_target_data[i, t - 1, word_int] = 1.0
                # the loop iterates over answers and if its a match for each word a alue of 1.0 is set correspondingly in decoder_target_data
    return encoder_input_data, decoder_input_data, decoder_target_data
#
encoder_input_train, decoder_input_train, decoder_target_train = preprocess_data(X_train, y_train, max_length)
encoder_input_test, decoder_input_test, decoder_target_test = preprocess_data(X_test, y_test, max_length)
# applying the preprocess_data function to training and testing of the data

In [69]:
#27  Train the model with a smaller batch size and gradient clipping
epochs = 50
batch_size = 32
optimizer = Adam(clipnorm=1.0) #optimizer is set as the adams optimmizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

model.fit([encoder_input_train, decoder_input_train], decoder_target_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
# training the model that we have created.

Epoch 1/50
254/254 [==============================] - 101s 376ms/step - loss: 2.1862 - val_loss: 2.0854
Epoch 2/50
254/254 [==============================] - 91s 357ms/step - loss: 2.0748 - val_loss: 2.0735
Epoch 3/50
254/254 [==============================] - 91s 358ms/step - loss: 2.0620 - val_loss: 2.0660
Epoch 4/50
254/254 [==============================] - 90s 355ms/step - loss: 2.0514 - val_loss: 2.0609
Epoch 5/50
254/254 [==============================] - 90s 353ms/step - loss: 2.0453 - val_loss: 2.0590
Epoch 6/50
254/254 [==============================] - 90s 353ms/step - loss: 2.0385 - val_loss: 2.0588
Epoch 7/50
254/254 [==============================] - 89s 351ms/step - loss: 2.0332 - val_loss: 2.0517
Epoch 8/50
254/254 [==============================] - 89s 351ms/step - loss: 2.0273 - val_loss: 2.0531
Epoch 9/50
254/254 [==============================] - 89s 351ms/step - loss: 2.0222 - val_loss: 2.0410
Epoch 10/50
254/254 [==============================] - 90s 354ms/step - 

In [70]:
#28  Test the model and evaluate its performance
loss = model.evaluate([encoder_input_test, decoder_input_test], decoder_target_test, batch_size=batch_size)
print('Test loss:', loss)


32/32 [==============================] - 4s 112ms/step - loss: 1.8815
Test loss: 1.8814653158187866


In [71]:
#29 (Optional) Save the trained modelb
model.save('seq2seq_chatbot.h5')

In [ ]:
#30 Load the saved model
from tensorflow.keras.models import load_model
model = load_model('seq2seq_chatbot.h5')

# Define functions for encoding and decoding input/output sequences
# the elow coode encodes the the input sequence by mapping each word to its integer representation.
def encode_input_sequence(input_sequence, word2int):
    encoded_sequence = [word2int['<SOS>']]
    for word in input_sequence.split():
        if word in word2int:
            encoded_sequence.append(word2int[word])
            # if the word is found the word is added to the encoded sequence.
        else:
            encoded_sequence.append(word2int['<OUT>'])
            # if the condition is unsatisfied then it is mapped to the <out> tooken
    return encoded_sequence

# the similar approach is used for the decoder sequence as well.
def decode_output_sequence(output_sequence, int2word):
    decoded_sequence = ''
    for integer in output_sequence:
        if integer > 0:
            decoded_sequence += int2word[integer] + ' '
    return decoded_sequence


# Define a function to generate a response from the chatbot
def generate_response(input_text):
    # Encode the input sequence
    input_sequence = encode_input_sequence(input_text, questionswords2int)
    # Pad the input sequence
    padded_input_sequence = pad_sequences([input_sequence], maxlen=max_length, padding='post')
    # Predict the output sequence
    output_sequence = model.predict([padded_input_sequence, np.zeros((1, max_length))])[0]
    # Decode the output sequence
    decoded_output_sequence = decode_output_sequence(np.argmax(output_sequence, axis=1), answersints2word)
    # Remove the <EOS> token from the output sequence
    if '<EOS>' in decoded_output_sequence:
        decoded_output_sequence = decoded_output_sequence[:decoded_output_sequence.index('<EOS>')]
    return decoded_output_sequence

# Start the chat
while True:
    input_text = input('You: ')
    if input_text.lower() == 'quit':
        break
    response = generate_response(input_text)
    print('Bot:', response)
